In [1]:
import pandas as pd
import numpy as np
import math
import toml
import os
import forecasting_tool


data_config = toml.load(os.path.join(os.getcwd(), "../configuration.toml"))


## Read from spreadsheet and create dataframe inputs


## PV Factor and CPI
   - used to canculate constant local transit revenue

In [2]:
parameter = pd.read_csv(data_config['parameter']).astype({'Year':'int64'})
parameter

,Year,PV factor,CPI,indecies
0,1975,5.312,0.5066,NaN
1,1976,5.047,0.5332,1.052507
2,1977,4.658,0.5778,1.083646
3,1978,4.241,0.6345,1.098131
4,1979,3.864,0.6965,1.097715
...,...,...,...,...
71,2046,0.605,4.4493,1.025941
72,2047,0.590,4.5578,1.024386
73,2048,0.578,4.6596,1.022335
74,2049,0.565,4.7590,1.021332


# Population
  - create function to calculate all year

In [3]:
subarea_population = pd.read_csv(data_config['data_subarea_population'])

result_subarea_population = pd.DataFrame()
for i_county in data_config['all_counties']:
    for i_pop_area in subarea_population.PopulationArea.unique():

        df_pop = subarea_population[
            (subarea_population['County'] == i_county) & (subarea_population['PopulationArea'] == i_pop_area)].copy()

        if df_pop.empty:
            pass

        else:
            start_year = int(df_pop['Year'].min())
            end_year = int(df_pop['Year'].max())
            # fill in missing years
            df_pop = forecasting_tool.fill_year(df_pop, ['County', 'PopulationArea'], start_year, end_year)
            # calculate interpolated population
            df_pop['Population'] = forecasting_tool.interpolate_population(df_pop, 'Population', start_year, end_year)
            df_pop = df_pop[['County', 'PopulationArea', 'Year', 'Population']]

            result_subarea_population = pd.concat([result_subarea_population, df_pop], ignore_index=True)

result_subarea_population

,County,PopulationArea,Year,Population
0,King County,Cities,2000,1387812.0
1,King County,Cities,2001,1404829.0
2,King County,Cities,2002,1427263.0
3,King County,Cities,2003,1436622.0
4,King County,Cities,2004,1443623.0
...,...,...,...,...
1015,Snohomish County,City Transit,2046,182008.0
1016,Snohomish County,City Transit,2047,184374.0
1017,Snohomish County,City Transit,2048,186772.0
1018,Snohomish County,City Transit,2049,189200.0


### Actual transit boardings (000s) from Local Transit Tab
- Year 2018 and before

In [4]:
# Local Transit - Total Fixed-Route boardings
transit_boardings = pd.read_csv(data_config['data_boardings_local_transit'])

result_boarding = pd.DataFrame()
for agency in data_config['county_transit']:

    df_agency = transit_boardings[(transit_boardings['Transit Agency'] == agency)].copy()

    # fill in missing years
    start_year = df_agency['Year'].min()

    df_agency = forecasting_tool.fill_year(df_agency, 'Transit Agency', start_year, data_config['end_year'])

    # last and next year with population values
    # Everett transit using City Transit Population for boarding estimation, others using PTBA
    if agency == "Everett Transit":
        df_agency1 = pd.merge(df_agency, result_subarea_population[
            (result_subarea_population['County'] == data_config['county_transit'][agency]) &
            (result_subarea_population['PopulationArea'] == 'City Transit')],
                              how="left", on="Year")
    else:
        df_agency1 = pd.merge(df_agency, result_subarea_population[
            (result_subarea_population['County'] == data_config['county_transit'][agency]) &
            (result_subarea_population['PopulationArea'] == 'PTBA')],
                              how="left", on="Year")

    # calculation
    for year in range(start_year, data_config['end_year'] + 1):
        miss = df_agency1.loc[df_agency1['Year'] == year, 'Boardings'].item()

        if np.isnan(miss):
            boarding_prev_year = df_agency1.loc[df_agency1['Year'] == year - 1, 'Boardings'].item()
            pop = df_agency1.loc[df_agency1['Year'] == year, 'Population'].item()
            pop_prev_year = df_agency1.loc[df_agency1['Year'] == year - 1, 'Population'].item()

            df_agency1.loc[df_agency1['Year'] == year, 'Boardings'] = boarding_prev_year * pop / pop_prev_year

        else:
            pass

    result_boarding = pd.concat([result_boarding, df_agency1], ignore_index=True)

result_boarding = result_boarding[['Transit Agency', 'Year', 'Boardings']]

result_boarding

,Transit Agency,Year,Boardings
0,Community Transit,1989,4.022714e+06
1,Community Transit,1990,4.138693e+06
2,Community Transit,1991,4.916127e+06
3,Community Transit,1992,5.057887e+06
4,Community Transit,1993,5.289549e+06
...,...,...,...
305,Pierce Transit,2046,1.390027e+07
306,Pierce Transit,2047,1.403806e+07
307,Pierce Transit,2048,1.417722e+07
308,Pierce Transit,2049,1.431775e+07


# Fare per Boarding


In [5]:
# Average Fixed-Route fare per boarding with periodic increases
fare_per_boarding = pd.read_csv(data_config["data_fare_per_boarding_local_transit"])

result_fare_per_boarding = pd.DataFrame()
for agency in data_config['county_transit']:
    fare_per_boarding1 = fare_per_boarding.copy()
    # get the last year with actual value for creating periodic increases
    last_value_year = max(fare_per_boarding1['Year'])
    # base list of fare change period
    list_grp = sorted(list(range(1,data_config['end_year']-last_value_year)) * data_config['transit_fare_change_period'][agency])

    fare_per_boarding1 = forecasting_tool.fill_year(fare_per_boarding1[fare_per_boarding1['Transit Agency']==agency], 'Transit Agency', 1989, data_config['end_year'])

    # Average Fixed-Route fare per boarding by Annual Growth Rate: This has the fare grown by the average annual growth rate for each transit agency based on previous 20 year history
    for year in range(1989, data_config['end_year'] + 1):

        miss = fare_per_boarding1.loc[fare_per_boarding1['Year'] == year, 'Average Fare per Boarding ($)'].item()

        if np.isnan(miss):

            fare_prev_year = fare_per_boarding1.loc[fare_per_boarding1['Year'] == year - 1, 'Average Fare per Boarding ($)'].item()

            fare_per_boarding1.loc[fare_per_boarding1['Year'] == year, 'Average Fare per Boarding ($)'] = fare_prev_year + (fare_prev_year * data_config['transit_annual_fare_increase'][agency])

        else:
            pass

    # add grouping column (trim base list to needed length)
    fare_per_boarding1['list_grp'] = list([0] * (last_value_year-1989)) + list_grp[0:data_config['end_year']-last_value_year+1]
    # Average Fixed-Route fare per boarding with periodic increases
    fare_per_boarding1.loc[fare_per_boarding1['list_grp']>0,'Average Fare per Boarding ($)'] = fare_per_boarding1.loc[fare_per_boarding1['list_grp']>0,['list_grp','Average Fare per Boarding ($)']].groupby(['list_grp'])['Average Fare per Boarding ($)'].transform(min)
    result_fare_per_boarding = pd.concat([result_fare_per_boarding, fare_per_boarding1[['Year', 'Transit Agency', 'Average Fare per Boarding ($)']]], ignore_index=True)
result_fare_per_boarding

,Year,Transit Agency,Average Fare per Boarding ($)
0,1989,Community Transit,0.811890
1,1990,Community Transit,0.854134
2,1991,Community Transit,0.880978
3,1992,Community Transit,0.952374
4,1993,Community Transit,0.984394
...,...,...,...
305,2046,Pierce Transit,2.934295
306,2047,Pierce Transit,2.934295
307,2048,Pierce Transit,3.489484
308,2049,Pierce Transit,3.489484


# Actual transit revenues (Nominal $millions) from Local Transit Tab


### Fare revenue calculation
- Year 2018 and before

In [6]:
transit_fare_revenue = pd.merge(result_boarding[['Year', 'Transit Agency', 'Boardings']],
                                result_fare_per_boarding[['Year', 'Transit Agency', 'Average Fare per Boarding ($)']],
                                on=['Year', 'Transit Agency'])
transit_fare_revenue['Nominal'] = transit_fare_revenue['Boardings'] * transit_fare_revenue['Average Fare per Boarding ($)']
transit_fare_revenue['Revenue Type'] = "Fares"
transit_fare_revenue = transit_fare_revenue[['Revenue Type', 'Transit Agency', 'Year', 'Nominal']]

transit_fare_revenue

,Revenue Type,Transit Agency,Year,Nominal
0,Fares,Community Transit,1989,3.266000e+06
1,Fares,Community Transit,1990,3.535000e+06
2,Fares,Community Transit,1991,4.331000e+06
3,Fares,Community Transit,1992,4.817000e+06
4,Fares,Community Transit,1993,5.207000e+06
...,...,...,...,...
305,Fares,Pierce Transit,2046,4.078749e+07
306,Fares,Pierce Transit,2047,4.119180e+07
307,Fares,Pierce Transit,2048,4.947117e+07
308,Fares,Pierce Transit,2049,4.996156e+07


In [42]:
transit_revenue = pd.read_csv(data_config['data_revenue_local_transit'])
transit_revenue

,Revenue Type,Transit Agency,Year,Nominal
0,Sales & Use Tax,Community Transit,1989,8088000.0
1,MVET,Community Transit,1989,8088000.0
2,Fares,Community Transit,1989,3266000.0
3,Sales & Use Tax,Everett Transit,1989,4177000.0
4,MVET,Everett Transit,1989,0.0
...,...,...,...,...
605,Non-PSRC FHWA,Pierce Transit,2020,0.0
606,PSRC FTA,Pierce Transit,2020,8351312.0
607,Non-PSRC FTA,Pierce Transit,2020,0.0
608,State,Pierce Transit,2020,0.0


## Local Transit Revenue: PSRC FHWA, PSRC FTA, Non-PSRC FTA, State, Other Federal

In [43]:
parameter = pd.read_csv(data_config['parameter']).astype({'Year':'int64'})

predict_list_other = ['Non-PSRC FTA','State','Other Federal']

process_list = ['PSRC FHWA', 'PSRC FTA']
max_year = max(transit_revenue['Year'])

result_transit_funding = pd.DataFrame()
for agency in data_config['county_transit'].keys():
    # forecasting ['PSRC FHWA', 'PSRC FTA'] fundings
    for predict in process_list:
        # first predicting year uses the average of the last five years with known revenue
        df = transit_revenue[(transit_revenue['Transit Agency'] == agency) & (transit_revenue['Revenue Type'] == predict)].copy()
        df = forecasting_tool.fill_year(df, ['Revenue Type', 'Transit Agency'],
                                             min(df['Year']), data_config['end_year'])
        df.loc[df['Year'] == max_year + 1, 'Nominal'] = df[df['Year'].isin(range(max_year-5, max_year+1))]['Nominal'].mean() * \
                data_config['psrc_transit_funding_annual_increase']
        # other years: previous year funding * 1.025
        for year in range(max_year + 2, data_config['end_year'] + 1):
            funding_prev_year = df.loc[df['Year'] == year - 1, 'Nominal'].item()
            df.loc[df['Year'] == year, 'Nominal'] = funding_prev_year * data_config['psrc_transit_funding_annual_increase']
        result_transit_funding = pd.concat([result_transit_funding, df], ignore_index=True)

    # forecasting ['Non-PSRC FTA','State','Other Federal'] fundings
    for predict in predict_list_other:
        # first predicting year uses the average of the last five years with known revenue
        df = transit_revenue[(transit_revenue['Transit Agency'] == agency) & (transit_revenue['Revenue Type'] == predict)].copy()
        df = forecasting_tool.fill_year(df, ['Revenue Type', 'Transit Agency'],
                                             min(df['Year']), data_config['end_year'])
        df.loc[df['Year'] == max_year + 1, 'Nominal'] = df[df['Year'].isin(range(max_year-5, max_year+1))]['Nominal'].mean() * \
                parameter.loc[parameter['Year']==max_year + 1,'indecies'].item()
        # other years: previous year funding * 1.025
        for year in range(max_year + 2, data_config['end_year'] + 1):
            funding_prev_year = df.loc[df['Year'] == year - 1, 'Nominal'].item()
            df.loc[df['Year'] == year, 'Nominal'] = funding_prev_year * parameter.loc[parameter['Year']==year,'indecies'].item()
        result_transit_funding = pd.concat([result_transit_funding, df], ignore_index=True)

result_transit_funding

# df_wide=pd.pivot(result_transit_funding, index=['Year','Transit Agency'], columns = 'Revenue Type',values = 'Nominal') #Reshape from long to wide
# df_wide

,Year,Revenue Type,Transit Agency,Nominal
0,2016,PSRC FHWA,Community Transit,608182.0
1,2017,PSRC FHWA,Community Transit,611209.0
2,2018,PSRC FHWA,Community Transit,619709.0
3,2019,PSRC FHWA,Community Transit,694099.0
4,2020,PSRC FHWA,Community Transit,713708.0
...,...,...,...,...
870,2046,Other Federal,Pierce Transit,0.0
871,2047,Other Federal,Pierce Transit,0.0
872,2048,Other Federal,Pierce Transit,0.0
873,2049,Other Federal,Pierce Transit,0.0


In [47]:
transit_sales_tax_rate = pd.read_csv(data_config['data_transit_sales_tax_rate']).astype({'Year':'int64'})

retail_sales = pd.read_csv(data_config['data_tax_base'])
retail_sales = retail_sales[retail_sales['Tax Base Category'] == 'Retail Sales (nominal)']

transit_revenue = pd.read_csv(data_config['data_revenue_local_transit'])

result_transit_sales_tax = pd.DataFrame()
for agency in data_config['county_transit'].keys():
    transit_sales_tax = transit_revenue[(transit_revenue['Transit Agency'] == agency) & (transit_revenue['Revenue Type'] == 'Sales & Use Tax')].copy()

    max_year = max(transit_sales_tax['Year'])
    test = min(transit_sales_tax['Year'])

    transit_sales_tax = forecasting_tool.fill_year(transit_sales_tax, ['Revenue Type', 'Transit Agency'],
                                         min(transit_sales_tax['Year']), data_config['end_year'])


    sales = pd.merge(retail_sales[retail_sales['County']==data_config['county_transit'][agency]],
                     transit_sales_tax_rate[['Year', agency]],
                     on=['Year'])
    sales['Sales & Use Tax'] = sales[agency] * sales['Value'] * data_config['transit_boundary_cal'][agency]

    transit_sales_tax = pd.merge(transit_sales_tax,sales[['Year','Sales & Use Tax']],
                                 how='left', on=['Year'])
    transit_sales_tax.loc[np.isnan(transit_sales_tax['Nominal']),'Nominal'] = transit_sales_tax['Sales & Use Tax']
    transit_sales_tax = transit_sales_tax[['Year','Revenue Type', 'Transit Agency', 'Nominal']]
    result_transit_sales_tax = pd.concat([result_transit_sales_tax, transit_sales_tax], ignore_index=True)

result_transit_sales_tax

,Year,Revenue Type,Transit Agency,Nominal
0,1989,Sales & Use Tax,Community Transit,8.088000e+06
1,1990,Sales & Use Tax,Community Transit,1.506800e+07
2,1991,Sales & Use Tax,Community Transit,1.834100e+07
3,1992,Sales & Use Tax,Community Transit,1.996400e+07
4,1993,Sales & Use Tax,Community Transit,2.106300e+07
...,...,...,...,...
305,2046,Sales & Use Tax,Pierce Transit,1.883062e+08
306,2047,Sales & Use Tax,Pierce Transit,1.948972e+08
307,2048,Sales & Use Tax,Pierce Transit,2.015230e+08
308,2049,Sales & Use Tax,Pierce Transit,2.080631e+08


In [ ]:
transit_revenue = pd.read_csv(data_config['data_revenue_local_transit'])

# calculate constant revenue with PV factor
transit_revenue = pd.merge(transit_revenue, parameter[['Year', 'PV factor']], how="left" , on="Year")
transit_revenue["Constant"] = transit_revenue['Nominal'] * transit_revenue['PV factor']
transit_revenue

### Annual Tax Base [Scenario: Base Model - Updated February, 2019]
- Year: 1975 - 2050


In [17]:
df = pd.read_csv("Excel_data/tax_base_actual.csv")
df

tax_base = pd.melt(df, id_vars=['County', 'Tax Base Category'], value_vars=df.columns[2:], var_name='Year',
                   value_name='Values').dropna()
# transit_boardings['Boardings'] = transit_boardings['Boardings (000s)'].apply(lambda x: x.strip().replace(',', '')).astype({'Boardings (000s)':float})*1e3
tax_base["Multiplier"] = 1e6
tax_base.loc[tax_base["Tax Base Category"].str.contains("000s"), "Multiplier"] = 1e3
tax_base.loc[tax_base["Tax Base Category"].str.contains("Diesel"), "Multiplier"] = 1
tax_base["True Value"] = tax_base["Multiplier"] * tax_base["Values"]
tax_base

,County,Tax Base Category,Year,Values,Multiplier,True Value
0,4-County regional Totals,Personal Income (nominal $mil),1975,13338.578310,1000000.0,1.333858e+10
1,4-County regional Totals,Population (000s),1975,1963.090000,1000.0,1.963090e+06
2,4-County regional Totals,Employment (000s),1975,719.238335,1000.0,7.192383e+05
3,4-County regional Totals,Retail Sales (nominal $mil),1975,7428.915395,1000000.0,7.428915e+09
4,4-County regional Totals,Motor Fuel (gal. Mil.),1975,941.502657,1000000.0,9.415027e+08
...,...,...,...,...,...,...
1819,Pierce County,Cars and Gas Trucks (000s),2050,1160.648093,1000.0,1.160648e+06
1820,Snohomish County,Population (000s),2050,1215.036446,1000.0,1.215036e+06
1821,Snohomish County,Employment (000s),2050,505.563009,1000.0,5.055630e+05
1822,Snohomish County,Retail Sales (nominal $mil),2050,37953.962990,1000000.0,3.795396e+10
